In [2]:
!pip install torch

    100% |████████████████████████████████| 519.5MB 28kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x59df0000 @  0x7fbf2b34c1c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import itertools
from collections import defaultdict

torch.manual_seed(7) # For reproducibility

In [0]:
next_index = 0
def get_new_index():
  global next_index
  index = next_index
  next_index += 1
  return index

vocabulary = defaultdict(get_new_index)

In [0]:
TRAIN_URL = 'https://raw.githubusercontent.com/pytorch/examples/master/word_language_model/data/wikitext-2/train.txt'
TRAIN_FILE = 'train.txt'
if not os.path.exists(TRAIN_FILE):
  import requests
  response = requests.get(TRAIN_URL)
  assert response.status_code == 200
  with open(TRAIN_FILE, 'wb') as f:
    f.write(response.content)

with open(TRAIN_FILE, 'r', encoding='utf-8') as f:
  content = [sentence.split() + ['<eos>'] for sentence in f.read().split(' . ')]
  all_words = list(itertools.chain.from_iterable(content))
  all_tokens = torch.tensor([vocabulary[word] for word in all_words])
  vocab_size = len(vocabulary)

In [0]:
class args:
  batch_size = 64
  seq_len = 32
  
device = torch.device('cuda')

In [0]:
def make_batched(tokens, batch_size):
  # See https://github.com/pytorch/examples/blob/master/word_language_model/main.py#L64-L83 for a comment
  data_size = len(tokens) // batch_size * batch_size
  tokens = tokens[:data_size]
  return tokens.reshape(batch_size, -1).t().contiguous()


def get_batch(data, i, seq_len):
  # See https://github.com/pytorch/examples/blob/master/word_language_model/main.py#L111-L125 for a comment
  seq_len = min(seq_len, len(data) - i - 1)
  samples = data[i:i + seq_len]
  targets = data[i + 1:i + 1 + seq_len]
  return samples, targets
  
train_data = make_batched(all_tokens, args.batch_size).to(device)

In [0]:
class LanguageModel(nn.Module):
  def __init__(self, embedding_size, hidden_size, num_layers):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_size)
    self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers)
    self.project = nn.Linear(hidden_size, vocab_size)
  
  def forward(self, input, hidden=None):
    embed = self.embedding(input)
    rnn_out, new_hidden = self.rnn(embed, hidden)
    return self.project(rnn_out), new_hidden
  
model = LanguageModel(128, 256, 2).to(device)
optimizer = torch.optim.Adam(model.parameters())

In [34]:
# NOTE: this cell only trains for a single epoch!
hidden = None
for i, offset in enumerate(range(0, len(train_data), args.seq_len)):
  samples, targets = get_batch(train_data, offset, args.seq_len)
  output, hidden = model(samples, hidden)
  hidden = tuple(h.detach() for h in hidden) # Truncated backpropagation through time
  loss = F.cross_entropy(output.reshape(-1, vocab_size),
                         targets.reshape(-1))
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  if i % 20 == 0:
    print(loss.item())

5.68550443649292
5.560492515563965
5.43204927444458
5.47991943359375
5.431942462921143
5.395793914794922
5.458574295043945
5.491985321044922
5.347511291503906
5.353490829467773
5.393863677978516
5.554957866668701
5.524961471557617
5.5842413902282715
5.476774215698242
5.497668266296387
5.560472011566162
5.522960186004639
5.446229457855225
5.542274475097656
5.444040775299072
5.297715187072754
5.384215831756592
5.311801910400391
5.374419212341309
5.489924430847168
5.308981895446777
5.3434014320373535
5.475695610046387
5.300534248352051
5.329865455627441
5.146168231964111
5.2738213539123535
5.419571399688721
5.351753234863281
5.357388496398926
5.332761764526367
5.408023834228516
5.387702465057373
5.241555213928223
5.349188327789307
5.347907066345215
5.4240007400512695
5.21343994140625
5.195337772369385
5.277040481567383
5.3807268142700195
5.343599796295166
5.40838098526001
5.399034023284912
5.302357196807861


In [57]:
num_generated_words = 10
reverse_vocab = {token: word for word, token in vocabulary.items()}
temperature = 1.
words = []
hidden = None
input = torch.randint(vocab_size, (1, 1), dtype=torch.long, device=device)

init_words = ['I', 'am']
for word in init_words:
  input = torch.tensor([[vocabulary[word]]], dtype=torch.long, device=device)
  output, hidden = model(input, hidden)
  words.append(word)

for i in range(num_generated_words):
  with torch.no_grad():
    word_probs = output.div(temperature).exp().view(-1)
    word_idx = torch.multinomial(word_probs, 1).item()
    input.fill_(word_idx)
    output, hidden = model(input, hidden)
    words.append(reverse_vocab[word_idx])
print(' '.join(words))

I am practices of her second interpretation , it Gives best centimetres
